In [ ]:
%%bash

wget https://sdk-for-java.amazonwebservices.com/latest/aws-java-sdk.zip
unzip aws-java-sdk.zip


In [3]:
%%bash
wget https://talend-update.talend.com/nexus/content/repositories/libraries/org/talend/libraries/hadoop-aws-2.8.3-amzn-1/6.0.0/hadoop-aws-2.8.3-amzn-1-6.0.0.jar

--2022-05-10 07:29:44--  https://talend-update.talend.com/nexus/content/repositories/libraries/org/talend/libraries/hadoop-aws-2.8.3-amzn-1/6.0.0/hadoop-aws-2.8.3-amzn-1-6.0.0.jar
Resolving talend-update.talend.com (talend-update.talend.com)... 149.97.185.22
Connecting to talend-update.talend.com (talend-update.talend.com)|149.97.185.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244841 (239K) [application/java-archive]
Saving to: ‘hadoop-aws-2.8.3-amzn-1-6.0.0.jar’

     0K .......... .......... .......... .......... .......... 20%  187K 1s
    50K .......... .......... .......... .......... .......... 41%  374K 1s
   100K .......... .......... .......... .......... .......... 62% 4.07M 0s
   150K .......... .......... .......... .......... .......... 83% 4.89M 0s
   200K .......... .......... .......... .........            100%  336K=0.5s

2022-05-10 07:29:46 (443 KB/s) - ‘hadoop-aws-2.8.3-amzn-1-6.0.0.jar’ saved [244841/244841]



In [2]:
%%bash 
ls |grep aws-java-sdk
ls aws-java-sdk-*/lib/
ls |grep hadoop-aws
pwd

aws-java-sdk-1.12.216
aws-java-sdk.zip
aws-java-sdk-1.12.216.jar
aws-java-sdk-1.12.216-javadoc.jar
aws-java-sdk-1.12.216-sources.jar
hadoop-aws-2.8.3-amzn-1-6.0.0.jar
/home/jovyan/work


In [5]:
!env |grep PYSPARK

|grep=PYSPARK
PYSPARK_SUBMIT_ARGS=--packages io.delta:delta-core_2.12:1.2.1 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell --packages=io.delta:delta-core_2.12:1.2.1,org.apache.hadoop:hadoop-aws:3.3.1
PYSPARK_PYTHONPATH_SET=1


In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.hadoop:hadoop-aws:3.3.1"
from pyspark.sql import SparkSession

tableName = "tbltestpython"

# Enable SQL/DML commands and Metastore tables for the current spark session.
# We need to set the following configs
# .config("spark.driver.extraClassPath", "/home/jovyan/work/aws-java-sdk-1.12.216/lib/aws-java-sdk-1.12.216.jar:/home/jovyan/work/hadoop-aws-2.8.3-amzn-1-6.0.0.jar").config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
# .config("spark.sql.catalogImplementation", "hive") \
# .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider") \
aws_key = 'hive'
aws_secret = '#202203sdf'
spark = SparkSession.builder.appName("quickstart_sql").master("local[*]") \
  .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
  .config("spark.hadoop.fs.s3a.path.style.access", True) \
  .config('spark.hadoop.fs.s3a.access.key', aws_key) \
  .config('spark.hadoop.fs.s3a.secret.key',  aws_secret) \
  .config('spark.hadoop.fs.s3a.endpoint', 'http://minio:9000') \
  .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
  .config("hive.metastore.warehouse.dir", "s3a://hive/") \
  .config("spark.sql.warehouse.dir", "s3a://hive/") \
  .enableHiveSupport().getOrCreate()


In [1]:
!pyspark --version


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.1
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.15
Branch HEAD
Compiled by user hgao on 2022-01-20T19:26:14Z
Revision 4f25b3f71238a00508a356591553f2dfa89f8290
Url https://github.com/apache/spark
Type --help for more information.


In [1]:
import os
os.environ['HADOOP_CONF_DIR'] = '/usr/local/spark/conf'
os.environ['PYSPARK_SUBMIT_ARGS'] =  "--packages=io.delta:delta-core_2.12:1.2.1,org.apache.hadoop:hadoop-aws:3.3.1   --conf spark.delta.logStore.class=org.apache.spark.sql.delta.storage.S3SingleDriverLogStore  --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell"

from pyspark.sql import SparkSession

tableName = "tbltestpython"

# Enable SQL/DML commands and Metastore tables for the current spark session.
# We need to set the following configs
# .config("spark.driver.extraClassPath", "/home/jovyan/work/aws-java-sdk-1.12.216/lib/aws-java-sdk-1.12.216.jar:/home/jovyan/work/hadoop-aws-2.8.3-amzn-1-6.0.0.jar").config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension").config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
# .config("spark.sql.catalogImplementation", "hive") \
# .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider") \
aws_key = 'hive'
aws_secret = '#202203sdf'
spark = SparkSession.builder.appName("quickstart_sql").master("local[*]") \
  .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
  .config("spark.hadoop.fs.s3a.path.style.access", True) \
  .config('spark.hadoop.fs.s3a.access.key', aws_key) \
  .config('spark.hadoop.fs.s3a.secret.key',  aws_secret) \
  .config('spark.hadoop.fs.s3a.endpoint', 'http://minio:9000') \
  .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
  .config("hive.metastore.warehouse.dir", "s3a://hive/") \
  .config("spark.sql.warehouse.dir", "s3a://hive/") \
  .enableHiveSupport().getOrCreate()

sc=spark.sparkContext
hadoop_conf=sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoop_conf.set("fs.s3n.awsAccessKeyId", aws_key)
hadoop_conf.set("fs.s3n.awsSecretAccessKey", aws_secret)

spark.sql("set hive.metastore.warehouse.dir=s3a://hive")
spark.sql("USE DEFAULT")
# Clear any previous runs
spark.sql("DROP TABLE IF EXISTS " + tableName)
spark.sql("DROP TABLE IF EXISTS newData")

try:
    # Create a table
    print("############# Creating a table ###############")
    spark.sql("CREATE TABLE %s(id LONG) USING delta" % tableName)
    spark.sql("INSERT INTO %s VALUES 0, 1, 2, 3, 4" % tableName)

    # Read the table
    print("############ Reading the table ###############")
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Upsert (merge) new data
    print("########### Upsert new data #############")
    spark.sql("CREATE TABLE newData(id LONG) USING parquet")
    spark.sql("INSERT INTO newData VALUES 3, 4, 5, 6")

    spark.sql('''MERGE INTO {0} USING newData
            ON {0}.id = newData.id
            WHEN MATCHED THEN
              UPDATE SET {0}.id = newData.id
            WHEN NOT MATCHED THEN INSERT *
        '''.format(tableName))

    spark.sql("SELECT * FROM %s" % tableName).show()

    # Update table data
    print("########## Overwrite the table ###########")
    spark.sql("INSERT OVERWRITE %s select * FROM (VALUES 5, 6, 7, 8, 9) x (id)" % tableName)
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Update every even value by adding 100 to it
    print("########### Update to the table(add 100 to every even value) ##############")
    spark.sql("UPDATE {0} SET id = (id + 100) WHERE (id % 2 == 0)".format(tableName))
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Delete every even value
    print("######### Delete every even value ##############")
    spark.sql("DELETE FROM {0} WHERE (id % 2 == 0)".format(tableName))
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Read old version of data using time travel
    print("######## Read old data using time travel ############")
    df = spark.read.format("delta").option("versionAsOf", 1).table(tableName)
    df.show()

finally:
    # cleanup
    spark.sql("DROP TABLE IF EXISTS " + tableName)
    spark.sql("DROP TABLE IF EXISTS newData")
    # spark.stop()

############# Creating a table ###############


Py4JJavaError: An error occurred while calling o49.sql.
: java.io.IOException: Cannot create file:/user/hive/warehouse/tbltestpython/_delta_log
	at org.apache.spark.sql.delta.DeltaLog.ensureLogDirectoryExist(DeltaLog.scala:338)
	at org.apache.spark.sql.delta.OptimisticTransactionImpl.prepareCommit(OptimisticTransaction.scala:639)
	at org.apache.spark.sql.delta.OptimisticTransactionImpl.prepareCommit$(OptimisticTransaction.scala:619)
	at org.apache.spark.sql.delta.OptimisticTransaction.prepareCommit(OptimisticTransaction.scala:98)
	at org.apache.spark.sql.delta.OptimisticTransactionImpl.liftedTree1$1(OptimisticTransaction.scala:547)
	at org.apache.spark.sql.delta.OptimisticTransactionImpl.$anonfun$commit$1(OptimisticTransaction.scala:545)
	at scala.runtime.java8.JFunction0$mcJ$sp.apply(JFunction0$mcJ$sp.java:23)
	at org.apache.spark.sql.delta.metering.DeltaLogging.recordFrameProfile(DeltaLogging.scala:120)
	at org.apache.spark.sql.delta.metering.DeltaLogging.recordFrameProfile$(DeltaLog

In [5]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
|     hive|
+---------+



In [3]:
spark.sql("show tables in default").show()

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|  default| sparktest|      false|
|  default|sparktest2|      false|
+---------+----------+-----------+



In [4]:
spark.read.orc("s3a://hive/test").show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+



In [3]:
spark.sql("show tables").show()

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|  default| sparktest|      false|
|  default|sparktest2|      false|
+---------+----------+-----------+



In [2]:
spark.sql("create or replace table default.sparktest13(id string)  using delta  location 's3a://hive/sparktest13' ")

Py4JJavaError: An error occurred while calling o49.sql.
: java.io.FileNotFoundException: No such file or directory: s3a://hive/sparktest13/_delta_log
	at org.apache.spark.sql.delta.storage.S3SingleDriverLogStore.listFromInternal(S3SingleDriverLogStore.scala:121)
	at org.apache.spark.sql.delta.storage.S3SingleDriverLogStore.exists(S3SingleDriverLogStore.scala:156)
	at org.apache.spark.sql.delta.storage.S3SingleDriverLogStore.write(S3SingleDriverLogStore.scala:174)
	at org.apache.spark.sql.delta.OptimisticTransactionImpl.doCommit(OptimisticTransaction.scala:881)
	at org.apache.spark.sql.delta.OptimisticTransactionImpl.doCommit$(OptimisticTransaction.scala:852)
	at org.apache.spark.sql.delta.OptimisticTransaction.doCommit(OptimisticTransaction.scala:98)
	at org.apache.spark.sql.delta.OptimisticTransactionImpl.$anonfun$doCommitRetryIteratively$2(OptimisticTransaction.scala:820)
	at org.apache.spark.sql.delta.metering.DeltaLogging.recordFrameProfile(DeltaLogging.scala:120)
	at org.apache.sp

In [7]:
spark.sql("update sparktest10 set id=id+100 where id%2 == 0")

AnalysisException: UPDATE destination only supports Delta sources.
Some(Relation default.sparktest10[id#13] parquet
)

In [2]:
spark.sql("create table sparktest10(id string)  using parquet location 's3a://hive/sparktest10'   ")

DataFrame[]

In [5]:
spark.sql("insert into sparktest10 values 1,2,3,4,5")

DataFrame[]

In [2]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|sparktest|      false|
+---------+---------+-----------+

